In [1]:
import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time

from google.colab import files
files.upload()

Saving trump_speeches.txt to trump_speeches.txt


{'trump_speeches.txt': b'Thank you so much.  That\'s so nice.  Isn\'t he a great guy.  He doesn\'t get a fair press; he doesn\'t get it.  It\'s just not fair.  And I have to tell you I\'m here, and very strongly here, because I have great respect for Steve King and have great respect likewise for Citizens United, David and everybody, and tremendous resect for the Tea Party.  Also, also the people of Iowa.  They have something in common.  Hard-working people.  They want to work, they want to make the country great.  I love the people of Iowa.  So that\'s the way it is.  Very simple.\nWith that said, our country is really headed in the wrong direction with a president who is doing an absolutely terrible job.  The world is collapsing around us, and many of the problems we\'ve caused.  Our president is either grossly incompetent, a word that more and more people are using, and I think I was the first to use it, or he has a completely different agenda than you want to know about, which coul

In [2]:
text = open('trump_speeches.txt').read()
#text = open(path_to_file).read()
vocab = sorted(set(text))
vocab_size = len(vocab)
print('Data has {} characters, {} unique characters'.format(len(text), len(vocab)))
print(text[:100])

# create character to index dictionary and idx to character numpy array for all unique characters
char_to_idx = {u: i for i, u in enumerate(vocab)}
print(char_to_idx)
print('The index for lowercase letter Q is:', char_to_idx['Q'])

# To decode the output of an RNN
idx_to_char = np.array(vocab)

# turn text into integers
int_text = np.array([char_to_idx[i] for i in text])
print('{} --- are mapped to --- > {}'.format(text[:17], int_text[:17]))

Data has 852417 characters, 92 unique characters
Thank you so much.  That's so nice.  Isn't he a great guy.  He doesn't get a fair press; he doesn't 
{'\n': 0, ' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, '&': 6, "'": 7, '(': 8, ')': 9, ',': 10, '-': 11, '.': 12, '/': 13, '0': 14, '1': 15, '2': 16, '3': 17, '4': 18, '5': 19, '6': 20, '7': 21, '8': 22, '9': 23, ':': 24, ';': 25, '=': 26, '?': 27, '@': 28, 'A': 29, 'B': 30, 'C': 31, 'D': 32, 'E': 33, 'F': 34, 'G': 35, 'H': 36, 'I': 37, 'J': 38, 'K': 39, 'L': 40, 'M': 41, 'N': 42, 'O': 43, 'P': 44, 'Q': 45, 'R': 46, 'S': 47, 'T': 48, 'U': 49, 'V': 50, 'W': 51, 'X': 52, 'Y': 53, 'Z': 54, '[': 55, ']': 56, '_': 57, 'a': 58, 'b': 59, 'c': 60, 'd': 61, 'e': 62, 'f': 63, 'g': 64, 'h': 65, 'i': 66, 'j': 67, 'k': 68, 'l': 69, 'm': 70, 'n': 71, 'o': 72, 'p': 73, 'q': 74, 'r': 75, 's': 76, 't': 77, 'u': 78, 'v': 79, 'w': 80, 'x': 81, 'y': 82, 'z': 83, 'é': 84, '–': 85, '—': 86, '‘': 87, '’': 88, '“': 89, '”': 90, '…': 91}
The index for lowercase letter

In [0]:
def split_input_target(data):
    input_eg = data[:-1]
    target_eg = data[1:]
    return input_eg, target_eg
  

def loss_func(real, preds):
    return tf.losses.sparse_softmax_cross_entropy(labels=real, logits=preds)

In [0]:
class Model(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, hidden_size):
        super(Model, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)

        self.lstm = tf.keras.layers.CuDNNLSTM(self.hidden_size, return_sequences=True, recurrent_initializer='glorot_uniform', stateful=True)

        self.output_layer = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        embedding = self.embedding(x)
        h = self.lstm(embedding)
        prediction = self.output_layer(h)
        return prediction

In [5]:
seq_len = 100
sections = tf.data.Dataset.from_tensor_slices(int_text).batch(seq_len+1, drop_remainder=True)
dataset = sections.map(split_input_target)

# create batches and shuffling them
batch_size = 64
# buffer_size is the number of elements from this dataset from which the new dataset will sample
buffer_size = 10000

hidden_size = 1024
embedding_dim = 256

rnn_lstm = Model(vocab_size, embedding_dim, hidden_size)
optimiser = tf.train.AdamOptimizer()
rnn_lstm.build(tf.TensorShape([batch_size, seq_len]))
rnn_lstm.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  23552     
_________________________________________________________________
cu_dnnlstm (CuDNNLSTM)       multiple                  5251072   
_________________________________________________________________
dense (Dense)                multiple                  94300     
Total params: 5,368,924
Trainable params: 5,368,924
Non-trainable params: 0
_________________________________________________________________


In [0]:
# use tf.train.Checkpoint to save the weights of the model after training
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
# Checkpoint instance
checkpoint = tf.train.Checkpoint(optimizer=optimiser, model=rnn_lstm)

In [0]:
epochs = 30
loss_list = []
for epoch in range(epochs):
    print('Epoch:', epoch + 1)
    data = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)
    rnn_lstm.reset_states()
    loss_per_epoch = []
    
    for (batch, (inputs, targets)) in enumerate(data):
        with tf.GradientTape() as tape:
            output = rnn_lstm(inputs)
            loss = loss_func(targets, output)
        grads = tape.gradient(loss, rnn_lstm.variables)
        optimiser.apply_gradients(zip(grads, rnn_lstm.variables))
        loss_per_epoch.append(loss)

        if batch % 100 == 0 and batch != 0:
           print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, batch, loss))
            
    loss_list.append(np.mean(loss_per_epoch))
    if (epoch+1) % 1 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)
      model = Model(vocab_size, embedding_dim, hidden_size)

      checkpoint = tf.train.Checkpoint(model=model)
      checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

      model.build(tf.TensorShape([1, None]))
      # Generate text from trained model
      print('Generated text after {} epoch training:'.format(epoch + 1))
      text_generated = []
      start_char = 'I'
      generating_len = 1000
      # convert the start string to an integer
      start_int = [char_to_idx[i] for i in start_char]
      start_int = tf.expand_dims(start_int, 0)
      
      # Low temperatures results in more predictable text.
      # Higher temperatures results in more surprising text.
      # Experiment to find the best setting.
      temperature = 0.5

      model.reset_states()
      for i in range(generating_len):
          predictions = model(start_int)
          predictions = tf.squeeze(predictions, 0)

          predictions = predictions / temperature
          pred_id = tf.multinomial(predictions, num_samples=1)[-1, 0].numpy()

          start_int = tf.expand_dims([pred_id], 0)
          text_generated.append(idx_to_char[pred_id])

      print(start_char + ''.join(text_generated))

Epoch: 1
Instructions for updating:
Use tf.cast instead.
Epoch 1 Batch 100 Loss 2.3137
Generated text after 1 epoch training:
Instructions for updating:
Use tf.random.categorical instead.
I
éd ang, you don the doing to wall the going at be hate, I are fan the met the lant alle the sare then wase the ping the and the pere thing the caike save the gad the hat eally verace thith they he the we rethe an the seand the son the war the and I dout the cof the wand gont thith it thored res onde the the the store the wat the the sale tho ron thes meat in the set and the Chis meve the beser fo and tha ken the hasting. The wave gout that be’s of the ping to but and on the will of to ment the sone gor I and the mere son that we thay And and to hant ant at they the and the wald you ve the meany to beve now an the arering thay wally wo wat the sere pout the the we wint the wand that the kant io has and the wave on the the the ke bering to of the seonde theret. The hat wa wand to mant be hachary and a

In [0]:
  import matplotlib.pyplot as plt
  
  plt.plot(loss_list)
  plt.title('Loss')
  plt.xlabel('Number of epochs')
  plt.ylabel('Loss')